# "Bin" statistics & histogram

Statistics "bin" are counted on the sample of a single continuous variable.
Firstly, it divides the range of values into bins (sequential, non-overlapping sections),
and then it counts the number of observations in each bin.
It's weighted, it means the weighted count for each bin is calculated
(each element within a bin counted along with its weight).
It's really important to carefully choose a bin-constructing method
(for example, by the exact number of bins or by their width).
This decision has a big impact on how the data is shown and studied.
It makes sure that the way the data is shown is natural to understand and gives a true picture of the information.

This notebook uses definitions from [DataFrame](https://kotlin.github.io/dataframe/overview.html).

## Usage

Binning is commonly used in statistics and data analysis
to simplify complex data sets and make them easier to interpret.
Histogram (or any other plot with "bin" statistics) helps to give an overview of the sample distribution.

## Arguments

* Input (mandatory):
    - `x` — numeric sample on which the statistics are calculated
* Weights (optional):
    - `weights` — set of weights of the same size as the input sample.
      `null` (by default) means all weights are equal to `1.0` and the weighted count is equal to the normal one
* Parameters (optional):
    - `binsOption: BinsOption` — specifies either the number of bins or their width:
        - `BinsOption.byNumber(n: Int)` — values are divided into `n` bins (bins width is derived)
        - `BinsOption.byWidth(width: Double)` — values are divided into bins of width `width`
          (the number of bins is derived)
    - `binsAlign: BinsAlign` — specifies bins aligning:
        - `BinsAlign.center(pos: Double)` — bins are aligned by centering bin in `pos`
        - `BinsAlign.boundary(pos: Double)` — bins are aligned by boundary between two bins in `pos`
        - `BinsAlign.none()` — no aligning

### Generalized signature

The specific signature depends on the function,
but all functions related to "bin" statistic
(which will be discussed further below - different variations of `statBin()`, `histogram()`)
have approximately the same signature with the arguments above:

```
statBinArgs := 
   x, 
   weights = null, 
   binsOption: BinsOption = BinsOption.byNumber(20), 
   binsAlign: BinsAlign = BinsAlign.center(0.0)
```

The possible types of `x` and `weights` depend on where a certain function is used.
They can be simply `Iterable` (`List`, `Set`, etc.) or a reference to a column in a `DataFrame`
(`String`, `ColumnAccessor`) or the `DataColumn` itself.

## Output statistics

| name                 | type   | description                                              |
|----------------------|--------|----------------------------------------------------------|
| Stat.x               | Double | Center of bin                                            |
| Stat.count           | Int    | Number of observations in this bin                       |
| Stat.countWeighted   | Double | Weighted count (sum of observations weights in this bin) |
| Stat.density         | Double | Empirically estimated density in this bin                |
| Stat.densityWeighted | Double | Weighted density                                         |

## StatBin plots

In [1]:
%useLatestDescriptors
%use kandy
%use dataframe

import org.apache.commons.math3.distribution.NormalDistribution
import org.apache.commons.math3.distribution.UniformRealDistribution
import org.jetbrains.kotlinx.statistics.dataframe.stat.mean

In [2]:
// Generate sample from normal distribution
val depthList = NormalDistribution(500.0, 100.0).sample(1000).toList()
// Generate sample from uniform distribution
val coeffList = UniformRealDistribution(0.0, 1.0).sample(1000).toList()
// gather them into the DataFrame
val df = dataFrameOf(
    "depth" to depthList,
    "coeff" to coeffList
)
df.head()

depth,coeff
"454,216715","0,256878"
"445,706876","0,167159"
"643,572508","0,604538"
"343,145749","0,625817"
"382,477519","0,808473"


`df` has a signature

| depth | coeff |
|-------|-------|

Let's take a look at `StatBin` output DataFrame:

In [3]:
df.statBin("depth", "coeff", binsOption = BinsOption.byNumber(10))

Stat,,,,
x,count,countWeighted,density,densityWeighted
"140,207366",2,"1,174281","0,000029","0,000034"
"210,311049",4,"1,702821","0,000057","0,000049"
"280,414732",24,"12,430909","0,000342","0,000357"
"350,518415",105,"50,634440","0,001498","0,001453"
"420,622098",219,"105,334214","0,003124","0,003022"
"490,725781",267,"132,978243","0,003809","0,003815"
"560,829463",221,"116,098474","0,003152","0,003331"
"630,933146",102,"50,639601","0,001455","0,001453"
"701,036829",43,"19,450610","0,000613","0,000558"


It has the following signature:

<table>
  <thead>
    <tr>
      <th alignt="left" colspan="5">Stat</th>
    </tr>
  </thead>
  <thead>
    <tr>
      <th>x</th>
      <th>count</th>
      <th>countWeighted</th>
      <th>density</th>
      <th>densityWeighted</th>
    </tr>
  </thead>
</table>

As you can see, we got a `DataFrame` with one `ColumnGroup` called `Stat` which contains several columns with statics.
For `statBin`, each row corresponds to one bin.
`Stat.x` is the column with the centers of the bins.
`Stat.count` contains the number of observations in the bin.
`Stat.countWeighted` — weighted version of `count`.
There are also `Stat.density` and `Stat.densityWeighted`.
They contain empirically estimated density (both normal and weighted)
of the sample in the points corresponding to the centers of bins.

`DataFrame` with "bin" statistics is called `StatBinFrame`

### `statBin` context transform

`statBin(statBinArgs) { /*new plotting context*/ }` modifies a plotting context — instead of original data
(no matter was it empty or not) new `StatBin` dataset (calculated on given arguments.
Inputs and weights can be provided as `Iterable` or as dataset column reference — by name as a `String`,
as a `ColumnReference` or as a `DataColumn`) is used inside a new context
(original dataset and primary context are not affected —
you can add layers using initial dataset outside the `statBin` context).
Since the old dataset is irrelevant, we cannot use references for its columns.
But we can refer to the new ones.
They are all contained in the `Stat` group and can be called inside the new context:

In [4]:
plot {
    statBin(depthList, binsAlign = BinsAlign.center(500.0)) {
        // new `StatBin` dataset here
        area {
            // use `Stat.*` columns for mappings
            x(Stat.x)
            y(Stat.count)
            fillColor = Color.RED
            alpha = 0.5
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="vOEatb"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[1.0,2.0,2.0,8.0,15.0,26.0,65.0,78.0,118.0,132.0,140.0,138.0,101.0,66.0,46.0,34.0,12.0,11.0,4.0,1.0],
"x":[149.48158533613673,184.53342680252302,219.58526826890937,254.6371097352957,289.688951201682,324.74079266806837,359.7926341344547,394.84447560084106,429.89631706722736,464.94815853361365,500.0,535.0518414663863,570.1036829327727,605.1555243991589,640.2073658655453,675.2592073319316,710.311048798318,745.3628902647043,780.4147317310906,815.4665731974769]
},
"sampling":"none",
"alpha":0.5,
"inherit_aes":false,
"position":"identity",
"geom":"area",
"fill":"#ee6666",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"spec_id":"2"
};
 var containerDiv = document.getElementById("vOEatb");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.275371983345238 338.68027210884355 L24.275371983345238 338.68027210884355 L49.82839512370863 336.36054421768705 L75.38141826407205 336.36054421768705 L100.93444140443547 322.4421768707483 L126.48746454479887 306.2040816326531 L152.0404876851623 280.687074829932 L177.5935108255257 190.2176870748299 L203.14653396588915 160.0612244897959 L228.69955710625254 67.27210884353741 L254.25258024661593 34.79591836734693 L279.8056033869793 16.23809523809524 L305.3586265273428 20.877551020408134 L330.91164966770623 106.70748299319726 L356.46467280806957 187.89795918367346 L382.017695948433 234.2925170068027 L407.5707190887964 262.12925170068024 L433.12374222915986 313.16326530612247 L458.67676536952325 315.4829931972789 L484.22978850988665 331.72108843537416 L509.78281165025004 338.68027210884355 L509.78281165025004 341.0 L484.22978850988665 341.0 L458.67676536952325 341.0 L433.12374222915986 341.0 L407.5707190887964 341.0 L382.017695948433 341.0 L356.46467280806957 341.0 L330.91164966770623 341.0 L305.3586265273428 341.0 L279.8056033869793 341.0 L254.25258024661593 341.0 L228.69955710625254 341.0 L203.14653396588915 341.0 L177.5935108255257 341.0 L152.0404876851623 341.0 L126.48746454479887 341.0 L100.93444140443547 341.0 L75.38141826407205 341.0 L49.82839512370863 341.0 L24.275371983345238 341.0 Z" fill="rgb(238,102,102)" stroke-width="1.0" fill-opacity="0.5019607843137255">
 
 
 
 <path d="M24.275371983345238 338.68027210884355 L24.275371983345238 338.68027210884355 L49.82839512370863 336.36054421768705 L75.38141826407205 336.36054421768705 L100.93444140443547 322.4421768707483 L126.48746454479887 306.2040816326531 L152.0404876851623 280.687074829932 L177.5935108255257 190.2176870748299 L203.14653396588915 160.0612244897959 L228.69955710625254 67.27210884353741 L254.25258024661593 34.79591836734693 L279.8056033869793 16.23809523809524 L305.3586265273428 20.877551020408134 L330.91164966770623 106.70748299319726 L356.46467280806957 187.89795918367346 L382.017695948433 234.2925170068027 L407.5707190887964 262.12925170068024 L433.12374222915986 313.16326530612247 L458.67676536952325 315.4829931972789 L484.22978

### Histogram layer

A histogram is a statistical chart that serves to visually approximate the distribution of a numerical variable.
It's a bar plot where each bar is representing a bin: its x coordinate is corresponding to bin range and y to count.
So basically, we can build a histogram with `statBin` as follows:

In [5]:
val statBinBarsPlot = df.plot {
    statBin("depth") {
        bars {
            x(Stat.x)
            y(Stat.count)
        }
    }
    layout.title = "`statBin` + `bars`"
}
statBinBarsPlot

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="cIx4Vx"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"`statBin` + `bars`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[140.20736586554528,175.25920733193163,210.31104879831798,245.36289026470428,280.41473173109057,315.4665731974769,350.51841466386327,385.5702561302496,420.62209759663597,455.67393906302226,490.72578052940855,525.7776219957949,560.8294634621813,595.8813049285676,630.9331463949538,665.9849878613402,701.0368293277265,736.0886707941129,771.1405122604992,806.1923537268855],
"count":[1.0,2.0,1.0,5.0,14.0,22.0,56.0,68.0,105.0,129.0,146.0,136.0,113.0,78.0,52.0,36.0,17.0,12.0,6.0,1.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"spec_id":"5"
};
 var containerDiv = document.getElementById("cIx4Vx");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 `statBin` + `bars` 
 
 
 
 
 count 
 
 
 
 
 x

But we can do it even faster with `histogram(statBinArgs)` method:

In [6]:
val histogramPlot = plot {
    histogram(depthList)
    layout.title = "`histogram`"
}
histogramPlot

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="wywDSt"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"`histogram`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[1.0,2.0,1.0,5.0,14.0,22.0,56.0,68.0,105.0,129.0,146.0,136.0,113.0,78.0,52.0,36.0,17.0,12.0,6.0,1.0],
"x":[140.20736586554528,175.25920733193163,210.31104879831798,245.36289026470428,280.41473173109057,315.4665731974769,350.51841466386327,385.5702561302496,420.62209759663597,455.67393906302226,490.72578052940855,525.7776219957949,560.8294634621813,595.8813049285676,630.9331463949538,665.9849878613402,701.0368293277265,736.0886707941129,771.1405122604992,806.1923537268855]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"spec_id":"8"
};
 var containerDiv = document.getElementById("wywDSt");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 `histogram` 
 
 
 
 
 count 
 
 
 
 
 x

Let's compare them:

In [7]:
plotGrid(listOf(statBinBarsPlot, histogramPlot))

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="JW9YXl"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"layout":{
"name":"grid",
"ncol":2,
"nrow":1,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"`statBin` + `bars`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[140.20736586554528,175.25920733193163,210.31104879831798,245.36289026470428,280.41473173109057,315.4665731974769,350.51841466386327,385.5702561302496,420.62209759663597,455.67393906302226,490.72578052940855,525.7776219957949,560.8294634621813,595.8813049285676,630.9331463949538,665.9849878613402,701.0368293277265,736.0886707941129,771.1405122604992,806.1923537268855],
"count":[1.0,2.0,1.0,5.0,14.0,22.0,56.0,68.0,105.0,129.0,146.0,136.0,113.0,78.0,52.0,36.0,17.0,12.0,6.0,1.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"spec_id":"12"
},{
"ggtitle":{
"text":"`histogram`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[1.0,2.0,1.0,5.0,14.0,22.0,56.0,68.0,105.0,129.0,146.0,136.0,113.0,78.0,52.0,36.0,17.0,12.0,6.0,1.0],
"x":[140.20736586554528,175.25920733193163,210.31104879831798,245.36289026470428,280.41473173109057,315.4665731974769,350.51841466386327,385.5702561302496,420.62209759663597,455.67393906302226,490.72578052940855,525.7776219957949,560.8294634621813,595.8813049285676,630.9331463949538,665.9849878613402,701.0368293277265,736.0886707941129,771.1405122604992,806.1923537268855]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"spec_id":"13"
}],
"kind":"subplots"
};
 var containerDiv = document.getElementById("JW9YXl");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 `statBin` + `bars` 
 
 
 
 
 count 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 `histogram` 
 
 
 
 
 count 
 
 
 
 
 x

These two plots are identical.
Indeed, `histogram` just uses `statBin` and `bars` and performs coordinates mappings under the hood.
And we can customize histogram layer: `histogram()` optionally opens a new context,
where we can configure bars (as in the usual context opened by `bars { ... }`) —
even change coordinate mappings from default ones.
`StatBin` dataset of histogram is also can be accessed here.

In [8]:
df.plot {
    histogram(depth, binsAlign = BinsAlign.center(500.0)) {
        // Change a column mapped on `y` to `Stat.density`
        y(Stat.density)
        // Filling color depends on `density` statistic
        fillColor(Stat.density) {
            scale = continuous(Color.YELLOW..Color.RED)
        }
        borderLine.color = Color.BLACK
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="sCEuZy"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#fac858",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density",
"fill":"density"
},
"stat":"identity",
"data":{
"density":[2.852917159741722E-5,5.705834319483444E-5,5.705834319483444E-5,2.2823337277933775E-4,4.279375739612583E-4,7.417584615328477E-4,0.0018543961538321193,0.002225275384598543,0.0033664422484952316,0.003765850650859073,0.003994084023638411,0.003937025680443577,0.0028814463313391395,0.0018829253254295365,0.001312341893481192,9.699918343121855E-4,3.423500591690066E-4,3.138208875715894E-4,1.1411668638966887E-4,2.852917159741722E-5],
"x":[149.48158533613673,184.53342680252302,219.58526826890937,254.6371097352957,289.688951201682,324.74079266806837,359.7926341344547,394.84447560084106,429.89631706722736,464.94815853361365,500.0,535.0518414663863,570.1036829327727,605.1555243991589,640.2073658655453,675.2592073319316,710.311048798318,745.3628902647043,780.4147317310906,815.4665731974769]
},
"color":"#000000",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
},{
"type":"float",
"column":"density"
}]
}
}],
"spec_id":"17"
};
 var containerDiv = document.getElementById("sCEuZy");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0.0000 
 
 
 
 
 
 
 0.0005 
 
 
 
 
 
 
 0.0010 
 
 
 
 
 
 
 0.0015 
 
 
 
 
 
 
 0.0020 
 
 
 
 
 
 
 0.0025 
 
 
 
 
 
 
 0.0030 
 
 
 
 
 
 
 0.0035 
 
 
 
 
 
 
 0.0040 
 
 
 
 
 
 
 
 
 density 
 
 
 
 
 depth 
 
 
 
 
 
 
 
 
 density 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.001 
 
 
 
 
 
 
 
 
 0.002 
 
 
 
 
 
 
 
 
 0.003

If we specify weights, `Stat.countWeighted` is mapped to `y` by default:

In [9]:
df.plot {
    // Count sample mean
    val mean = depth.mean()
    // Add weighted histogram
    histogram(depth, coeff, binsOption = BinsOption.byNumber(10), binsAlign = BinsAlign.boundary(mean))
    // We can add other layers as well.
    // Let's add a vertical mark line in the mean of sample
    vLine {
        xIntercept.constant(mean)
        tooltips { line("Depth mean: ${String.format("%.2f", mean)}m") }
        color = Color.RED; width = 3.0
    }
    x.axis.name = "depth, m"
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="1XHbjf"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"depth, m",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"countWeighted"
},
"stat":"identity",
"data":{
"x":[181.43462686122646,251.53830979399913,321.64199272677183,391.7456756595444,461.8493585923171,531.9530415250897,602.0567244578624,672.1604073906351,742.2640903234078,812.3677732561804],
"countWeighted":[1.7105216671158094,5.172825674154362,32.48370622950262,73.97783994229292,135.42493253466608,128.18544332883445,81.41563514031046,28.31436207027674,9.746106273797826,0.7237936422676516]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"float",
"column":"countWeighted"
}]
}
},{
"mapping":{
},
"stat":"identity",
"color":"#ee6666",
"size":3.0,
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"xintercept":496.9012000587034,
"geom":"vline",
"tooltips":{
"lines":["Depth mean: 496,90m"],
"disable_splitting":true
},
"data":{
}
}],
"spec_id":"20"
};
 var containerDiv = document.getElementById("1XHbjf");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 countWeighted 
 
 
 
 
 depth, m

### `histogram` plot

`histogram(statBinArgs)` and `DataFrame.histogram(statBinArgs)` are a family of functions for fast plotting a histogram.

In [10]:
histogram(depthList, binsAlign = BinsAlign.center(500.0))

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Hky7jB"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[149.48158533613673,184.53342680252302,219.58526826890937,254.6371097352957,289.688951201682,324.74079266806837,359.7926341344547,394.84447560084106,429.89631706722736,464.94815853361365,500.0,535.0518414663863,570.1036829327727,605.1555243991589,640.2073658655453,675.2592073319316,710.311048798318,745.3628902647043,780.4147317310906,815.4665731974769],
"count":[1.0,2.0,2.0,8.0,15.0,26.0,65.0,78.0,118.0,132.0,140.0,138.0,101.0,66.0,46.0,34.0,12.0,11.0,4.0,1.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"spec_id":"23"
};
 var containerDiv = document.getElementById("Hky7jB");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 x

In [11]:
df.histogram("depth")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="rJA2kT"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[140.20736586554528,175.25920733193163,210.31104879831798,245.36289026470428,280.41473173109057,315.4665731974769,350.51841466386327,385.5702561302496,420.62209759663597,455.67393906302226,490.72578052940855,525.7776219957949,560.8294634621813,595.8813049285676,630.9331463949538,665.9849878613402,701.0368293277265,736.0886707941129,771.1405122604992,806.1923537268855],
"count":[1.0,2.0,1.0,5.0,14.0,22.0,56.0,68.0,105.0,129.0,146.0,136.0,113.0,78.0,52.0,36.0,17.0,12.0,6.0,1.0]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"spec_id":"26"
};
 var containerDiv = document.getElementById("rJA2kT");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 depth

In case you want to provide inputs and weights using column selection DSL, it's a bit different from the usual one —
you should assign `x` input and (optionally) `weight` throw invocation eponymous functions:

In [12]:
df.histogram(binsOption = BinsOption.byNumber(10)) {
    x(depth)
    weight(coeff)
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="zsUwRM"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"countWeighted"
},
"stat":"identity",
"data":{
"x":[140.2073658655453,210.31104879831798,280.4147317310906,350.51841466386327,420.62209759663597,490.72578052940867,560.8294634621814,630.9331463949538,701.0368293277265,771.1405122604992],
"countWeighted":[1.1742809762967588,1.702821092801275,12.430909437203919,50.63443994333093,105.33421416192023,132.9782426911781,116.0984742170615,50.639601271518586,19.450609623826253,6.711573088081292]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"float",
"column":"countWeighted"
}]
}
}],
"spec_id":"29"
};
 var containerDiv = document.getElementById("zsUwRM");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 
 
 countWeighted 
 
 
 
 
 depth

Histogram plot can be configured with `.configure {}` extension —
it opens a context that combines bars, `StatBin` and plot context.
That means you can configure bars settings, mappings using `StatBin` dataset and any plot adjustments:

In [13]:
df.histogram(binsOption = BinsOption.byNumber(15)) {
    x(depth)
}.configure {
    // Bars + StatBin + PlotBuilder
    // Can't add a new layer
    x.axis.limits = 100..900
    // Can add bar mapping, include on `Stat.*` columns
    fillColor(Stat.count) { scale = continuous(Color.GREEN..Color.RED) }
    // Can configure general plot adjustments
    layout {
        title = "Configured histogram plot"
        size = 600 to 350
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ke4Kx1"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Configured histogram plot"
},
"mapping":{
},
"coord":{
"name":"cartesian",
"xlim":[100.0,900.0],
"flip":false
},
"data":{
"x":[140.2073658655453,186.94315448739377,233.6789431092422,280.4147317310906,327.15052035293905,373.8863089747875,420.6220975966359,467.35788621848434,514.0936748403328,560.8294634621811,607.5652520840297,654.3010407058782,701.0368293277265,747.7726179495751,794.5084065714234],
"count":[1.0,2.0,6.0,17.0,44.0,85.0,148.0,180.0,184.0,150.0,88.0,53.0,25.0,13.0,4.0]
},
"ggsize":{
"width":600.0,
"height":350.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#3ba272",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"count"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
},
"spec_id":"32"
};
 var containerDiv = document.getElementById("ke4Kx1");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 350.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 
 
 Configured histogram plot 
 
 
 
 
 count 
 
 
 
 
 depth 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 150

## Grouped `staBin`

`statBin` can be applied for grouped data — statistics will be counted on each group independently but with equal bins.
This application returns a new `GroupBy`
dataset with the same keys as the old one but with `StatBin` groups instead of old ones.

In [14]:
// Create two samples from normal distribution with different mean/std
val rangesA = NormalDistribution(500.0, 100.0).sample(5000).toList()
val rangesB = NormalDistribution(400.0, 80.0).sample(5000).toList()

// Gather them into `DataFrame` with "A" and "B" keys in the "category" column
val rangesDF = dataFrameOf(
    "range" to rangesA + rangesB,
    "category" to List(5000) { "A" } + List(5000) { "B" }
)
rangesDF.head(5)

range,category
"407,318197",A
"420,695210",A
"586,104956",A
"404,210034",A
"496,729205",A


It has the following signature:

| range | category |
|-------|----------|

In [15]:
// Group it by "category"
val groupedRangesDF = rangesDF.groupBy { category }
groupedRangesDF

<iframe onload="o_resize_iframe_out_6()" style="width:100%;" class="result_container" id="iframe_out_6" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
 white-space: nowrap;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_1979711498"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.lengt

Now we have a `GroupBy` with a signature

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: DataFrame[range|category]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

In [16]:
groupedRangesDF.statBin { x(range) }

<iframe onload="o_resize_iframe_out_8()" style="width:100%;" class="result_container" id="iframe_out_8" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
 white-space: nowrap;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_1979711506"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.lengt

After `statBin` applying it's still a `GroupBy` but with different signature of `group` —
all groups have the same signature as usual `DataFrame` after `statBin` applying (i.e. `StatBinFrame`):

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: StaBinFrame</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

As you can see, we did indeed do a `statBin` transformation within groups, the grouping keys did not change.
Also, all bin centers match — it helps to build grouped histogram.

The plotting process doesn't change much — we do everything the same.

In [17]:
groupedRangesDF.plot {
    statBin(range) {
        area {
            x(Stat.x)
            y(Stat.density)
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="SiYu6Y"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"density":[0.0,5.368715135089618E-6,2.1474860540358472E-5,9.12681572965235E-5,2.684357567544809E-4,5.529776589142306E-4,0.001207960905395164,0.002066955327009503,0.0031782793599730535,0.003817156461048718,0.004413083841043665,0.00391916204861542,0.0030225866210554547,0.002158223484306026,0.0011918547599898952,5.959273799949476E-4,2.254860356737639E-4,5.9055866485985796E-5,2.6843575675448087E-5,2.1474860540358472E-5,6.582113573696161E-6,3.29105678684808E-5,1.316422714739232E-4,5.134048587483005E-4,0.001487557667655332,0.002994861676031753,0.005272272972530625,0.006200350986421784,0.006384650166485276,0.004442926662244909,0.0031067576067845878,0.0016323641662766477,5.134048587483005E-4,1.5797072576870783E-4,2.6328454294784645E-5,6.582113573696161E-6,0.0,0.0,0.0,0.0],
"x":[100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757,100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"float",
"column":"x"
},{
"type":"float",
"column":"density"
},{
"type":"str",
"column":"&merged_groups"
}]
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"35"
};
 var containerDiv = document.getElementById("SiYu6Y");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M23.68200399403556 340.6651939126166 L23.68200399403556 340.6651939126166 L48.61042925091508 339.325969563083 L73.53885450779461 334.30387825233186 L98.46727976467415 314.88512518409425 L123.39570502155368 265.33382425135005 L148.32413027843322 188.66323024054984 L173.25255553531272 72.820324005891 L198.18098079219226 25.612665684830574 L223.10940604907177 16.23809523809524 L248.03783130595133 115.00589101620028 L272.96625656283084 182.9715267550319 L297.89468181971034 257.9680903289151 L322.82310707658985 314.88512518409425 L347.7515323334694 332.9646539027982 L372.679957590349 339.66077565046635 L397.6083828472285 340.6651939126166 L422.536808104108 341.0 L447.4652333609875 341.0 L472.3936586

As you can see, there are two areas because we have two groups of data.
To distinguish them, we need to add mapping to the filling color from the key.
This is convenient — the key is available in the context

In [18]:
groupedRangesDF.plot {
    statBin(range) {
        area {
            x(Stat.x)
            y(Stat.density)
            // can access "key." columns and create mapping from them
            fillColor(key.category)
            alpha = 0.6
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="WR4zI8"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"density":[0.0,5.368715135089618E-6,2.1474860540358472E-5,9.12681572965235E-5,2.684357567544809E-4,5.529776589142306E-4,0.001207960905395164,0.002066955327009503,0.0031782793599730535,0.003817156461048718,0.004413083841043665,0.00391916204861542,0.0030225866210554547,0.002158223484306026,0.0011918547599898952,5.959273799949476E-4,2.254860356737639E-4,5.9055866485985796E-5,2.6843575675448087E-5,2.1474860540358472E-5,6.582113573696161E-6,3.29105678684808E-5,1.316422714739232E-4,5.134048587483005E-4,0.001487557667655332,0.002994861676031753,0.005272272972530625,0.006200350986421784,0.006384650166485276,0.004442926662244909,0.0031067576067845878,0.0016323641662766477,5.134048587483005E-4,1.5797072576870783E-4,2.6328454294784645E-5,6.582113573696161E-6,0.0,0.0,0.0,0.0],
"x":[100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757,100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"alpha":0.6,
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"float",
"column":"x"
},{
"type":"float",
"column":"density"
},{
"type":"str",
"column":"&merged_groups"
}]
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"38"
};
 var containerDiv = document.getElementById("WR4zI8");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M19.95662217490098 340.6651939126166 L19.95662217490098 340.6651939126166 L40.963592885323045 339.325969563083 L61.97056359574514 334.30387825233186 L82.9775343061672 314.88512518409425 L103.98450501658931 265.33382425135005 L124.99147572701139 188.66323024054984 L145.99844643743344 72.820324005891 L167.00541714785552 25.612665684830574 L188.0123878582776 16.23809523809524 L209.01935856869972 115.00589101620028 L230.02632927912177 182.9715267550319 L251

The `histogram` layer also works.
Moreover, if we have exactly one grouping key, a mapping from it to `fillColor` will be created by default.

In [19]:
groupedRangesDF.plot {
    histogram(range)
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="8eCBhU"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"count":[0.0,1.0,4.0,17.0,50.0,103.0,225.0,385.0,592.0,711.0,822.0,730.0,563.0,402.0,222.0,111.0,42.0,11.0,5.0,4.0,1.0,5.0,20.0,78.0,226.0,455.0,801.0,942.0,970.0,675.0,472.0,248.0,78.0,24.0,4.0,1.0,0.0,0.0,0.0,0.0],
"x":[100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757,100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
},{
"type":"str",
"column":"&merged_groups"
}]
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"41"
};
 var containerDiv = document.getElementById("8eCBhU");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 range 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

We can customize it like we used to.
From the differences - access to `key` columns, and we can customize the `position`
of bars (within a single x-coordinate), for example — stack them:

In [20]:
groupedRangesDF.plot {
    histogram(range) {
        fillColor(key.category) {
            scale = categorical(listOf(Color.GREEN, Color.ORANGE))
        }
        borderLine.width = 0.0
        width = 1.0
        // Adjust position of bars from different groups
        position = Position.stack()
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="cIv7rh"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#3ba272","#fc8452"]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"count":[0.0,1.0,4.0,17.0,50.0,103.0,225.0,385.0,592.0,711.0,822.0,730.0,563.0,402.0,222.0,111.0,42.0,11.0,5.0,4.0,1.0,5.0,20.0,78.0,226.0,455.0,801.0,942.0,970.0,675.0,472.0,248.0,78.0,24.0,4.0,1.0,0.0,0.0,0.0,0.0],
"x":[100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757,100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"size":0.0,
"sampling":"none",
"width":1.0,
"inherit_aes":false,
"position":"stack",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
},{
"type":"str",
"column":"&merged_groups"
}]
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"44"
};
 var containerDiv = document.getElementById("cIv7rh");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 range 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

Histogram plot for `GroupBy` (i.e. `GroupBy.histogram(statBinArgs)` extensions) works as well:

In [21]:
groupedRangesDF.histogram("range")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="1TkVii"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757,100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757],
"count":[0.0,1.0,4.0,17.0,50.0,103.0,225.0,385.0,592.0,711.0,822.0,730.0,563.0,402.0,222.0,111.0,42.0,11.0,5.0,4.0,1.0,5.0,20.0,78.0,226.0,455.0,801.0,942.0,970.0,675.0,472.0,248.0,78.0,24.0,4.0,1.0,0.0,0.0,0.0,0.0],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
},{
"type":"str",
"column":"&merged_groups"
}]
}
}],
"spec_id":"47"
};
 var containerDiv = document.getElementById("1TkVii");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 range 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

... and can be configured the same way:

In [22]:
groupedRangesDF.histogram(binsAlign = BinsAlign.center(500.0)) { x(range) }.configure {
    alpha = 0.6
    // make the bars from different groups overlap with each other
    position = Position.identity()
    // can access key column by name as `String`
    fillColor("category") { scale = categoricalColorBrewer(BrewerPalette.Qualitative.Dark2) }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="VNH0pw"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[99.3260462054532,139.39344158490786,179.46083696436256,219.52823234381722,259.59562772327195,299.6630231027266,339.7304184821812,379.797813861636,419.8652092410906,459.93260462054536,500.0,540.0673953794546,580.1347907589094,620.202186138364,660.2695815178188,700.3369768972734,740.404372276728,780.4717676561827,820.5391630356374,860.6065584150922,99.3260462054532,139.39344158490786,179.46083696436256,219.52823234381722,259.59562772327195,299.6630231027266,339.7304184821812,379.797813861636,419.8652092410906,459.93260462054536,500.0,540.0673953794546,580.1347907589094,620.202186138364,660.2695815178188,700.3369768972734,740.404372276728,780.4717676561827,820.5391630356374,860.6065584150922],
"count":[0.0,1.0,4.0,17.0,48.0,99.0,224.0,387.0,586.0,701.0,824.0,736.0,564.0,410.0,223.0,113.0,42.0,12.0,5.0,4.0,1.0,5.0,20.0,77.0,221.0,444.0,798.0,940.0,978.0,667.0,486.0,253.0,81.0,24.0,4.0,1.0,0.0,0.0,0.0,0.0],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_brewer",
"palette":"Dark2"
},{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"alpha":0.6,
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"50"
};
 var containerDiv = document.getElementById("VNH0pw");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 range 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

### Inside `groupBy{}` plot context

We can apply `groupBy` modification to the initial dataset and build a histogram with grouped data the same way:

In [23]:
rangesDF.plot {
    groupBy(category) {
        histogram(range)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="BqPCqg"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757,100.16848844863671,140.23588382809137,180.30327920754607,220.37067458700074,260.43806996645543,300.5054653459101,340.57286072536476,380.64025610481946,420.70765148427415,460.77504686372885,500.84244224318354,540.9098376226382,580.9772330020928,621.0446283815476,661.1120237610023,701.179419140457,741.2468145199116,781.3142098993662,821.381605278821,861.4490006582757],
"count":[0.0,1.0,4.0,17.0,50.0,103.0,225.0,385.0,592.0,711.0,822.0,730.0,563.0,402.0,222.0,111.0,42.0,11.0,5.0,4.0,1.0,5.0,20.0,78.0,226.0,455.0,801.0,942.0,970.0,675.0,472.0,248.0,78.0,24.0,4.0,1.0,0.0,0.0,0.0,0.0],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"category"
},{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
},{
"type":"str",
"column":"&merged_groups"
}]
}
}],
"spec_id":"53"
};
 var containerDiv = document.getElementById("BqPCqg");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 range 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B